In [0]:
%run /Repos/a.torralbo@ucl.ac.uk/ccu013_02/atlas/00_Load_helpers

Outline: 

1) N people, n admissions per phecode at baseline: ccu013_phedatav2_n_n_admi_baseline

2) Median duration of hospitalisation (admission): ccu013_admi_median_duration_before_baseline

3) Risk of hospitalisation 2020-2021 (to edit)

Then these tables (1 and 2) are exported as results through R (see code there)

In [0]:
%sql 
select count(*) from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort_sex_dep;

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort_sex_dep where person_admi_id == ''

n cases and n admissions - before baseline 23-01-2020 

In [0]:
phe_n_n_baseline = spark.sql("""
select phecode, count(distinct(person_id_deid)) as n_patients_before_baseline, count(distinct(person_admi_id)) as n_admissions_before_baseline
from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort_sex_dep
where phecode not like 'XROOT'
and date <= to_date('2020-01-23', 'yyyy-MM-dd')
group by phecode
sort by n_admissions_before_baseline desc""")

In [0]:
%sql 
select max(date) from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort_sex_dep;

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort_sex_dep limit 4;

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first_sex_dep limit 5;

In [0]:
phe_n_n_baseline.createOrReplaceTempView('ccu013_phedatav200_n_n_admi_baseline')

In [0]:
phe_n_n_baseline.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_phedatav200_n_n_admi_baseline')

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_phedatav200_n_n_admi_baseline limit 10;

n days duration of the admission 

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort_sex_dep limit 10;

In [0]:
admi_dur = spark.sql('''
            select distinct(person_admi_id), phecode, admi_dur
            from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort_sex_dep
            where phecode NOT LIKE '%ROOT' and date <= to_date('2020-01-23')
            group by phecode, person_admi_id, admi_dur
          ''')

In [0]:
%sql 
select distinct(person_admi_id), phecode, admi_dur 
          from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort_sex_dep
          where phecode NOT LIKE '%ROOT' and date <= to_date('2020-01-23')
          group by phecode, person_admi_id, admi_dur;

In [0]:
admi_dur.write.mode("overwrite").saveAsTable('dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_duration_before_baseline')

In [0]:
admi_median_dur_interval =spark.sql(''' 
select phecode, median(admi_dur) as median_duration, percentile(admi_dur, 0.25) as admi_dur_percentile_25, percentile(admi_dur, 0.75) as admi_dur_percentile_75
from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_duration_before_baseline
group by phecode''')

In [0]:
admi_median_dur_interval.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_median_duration_before_baseline')

In [0]:
admi_X401_1=spark.sql("""
select distinct(person_admi_id), phecode, admi_dur 
from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort_sex_dep 
where phecode == 'X401_1' and date <= to_date('2020-01-23', 'yyyy-MM-dd')
group by phecode, person_admi_id, admi_dur 
order by admi_dur desc """)

In [0]:
admi_X401_1.display()

In [0]:
import pyspark.sql.functions as f

admi_X401_1.agg(f.max('admi_dur')).show()

admissions after Jan 2020 baseline - admissions within one year (to calculate risk of hospitalisation) 

ON GOING - TO DO - FIX : 
add flag for cases pre2020 and restrict to these 
to make sure cohort is same cohort but their events 2020-2021.

Do not capture all the events upto 2021 incuding new cases, 
as then proportion can be greater than 100%

In [0]:
adm_1yafter_baseline = spark.sql("""
select phecode, count(distinct(person_id_deid)) as n_individuals_hosp_after_baseline
from dsa_391419_j3w9t_collab.ccu013_phedata200_admi_unrolled_cohort_sex_dep 
person_id_deid in (
  select person_id_deid 
  from dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_cohort_sex_dep t
  inner join dsa_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort_eth s
  on t.person_id_deid = s.NHS_NUMBER_DEID

)
where date > to_date('2020-01-23', 'yyyy-MM-dd') and date <= to_date('2021-01-21', 'yyyy-MM-dd')
group by phecode""")

In [0]:
adm_1yafter_baseline.createOrReplaceTempView('ccu013_phedatav200_admi_n_1y_afterbaseline')

In [0]:
adm_1yafter_baseline.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_n_1y_afterbaseline')

In [0]:
%sql 
select * from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_n_1y_afterbaseline limit 10;

In [0]:
n_icd_admi = spark.sql('''
select person_admi_id, count(distinct(DIAG_4)) as n_icd10_admi,
case
    when date >= to_date('1998-01-01') and date <= ('1999-12-31') then '1998-1999'
    when date >= to_date('2000-01-01') and date <= ('2004-12-31') then '2000-2004'
    when date >= to_date('2005-01-01') and date <= ('2009-12-31') then '2005-2009'
    when date >= to_date('2010-01-01') and date <= ('2019-12-31') then '2010-2019'
    when date >= to_date('2020-01-01') and date <= ('2021-12-31') then '2020-2021'
    end 
    as date_band 
from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort_sex_dep
where phecode NOT LIKE '%ROOT'
group by person_admi_id, date_band''')

In [0]:
%sql
drop table dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_n_icd10;

In [0]:
n_icd_admi.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_n_icd10')